Julia code for Kitaev model.
\section{Kitaev model without symmetry}

In [8]:
using SparseArrays
using Arpack
using BenchmarkTools

function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    #Calculate the index of the atoms in the (r,c) unit cell
    #A-sublattice atom = 0; B-subkattice: atom = 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
    end;

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
    end;
    
function StateConfig(tag::Int, N1::Int, N2::Int)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
    end;

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
    end;

function KitaevRhom(J::Array{Float64,1}, N1::Int, N2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice
    Inputes: N1, N2: column and row numbers
             J: coupling constants
    Outpus: The Non-zero Hamiltonian element H[i] is in the 
            position (row[i], col[i])
            H, row, col : type: list =#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    Ham = spzeros(Float64, dim, dim)

    for tag = 0: dim-1

        for r = 0: N2-1
            for c = 0: N1-1
                ind = index(N1, N2, c, r, 0)
                
                # Sx-Sx
                next = index(N1, N2, c, r, 1)
                ntag = flip(tag, ind, next, N1, N2)
                Ham[ntag+1, tag+1] += -J[1] /4
                
                # Sy-Sy
                next = index(N1, N2, c - 1, r, 1) 
                ntag = flip(tag, ind, next, N1, N2) 
                Ham[ntag+1, tag+1] += J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                        
                #Sz-Sz
                next = index(N1, N2, c, r - 1, 1)
                Ham[tag+1, tag+1] += - J[3] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
            end
        end                       
    end
    return Ham
    end;

In [9]:
N1 = 2; N2 = 3
J = -[1.0,1.0,1.0]

@benchmark H = KitaevRhom(J, N1, N2)    

BenchmarkTools.Trial: 
  memory estimate:  2.03 MiB
  allocs estimate:  35
  --------------
  minimum time:     15.290 ms (0.00% GC)
  median time:      16.223 ms (0.00% GC)
  mean time:        16.547 ms (0.00% GC)
  maximum time:     27.522 ms (0.00% GC)
  --------------
  samples:          297
  evals/sample:     1

Find a better way to generate CSC matrix
1. better be generated by columns
2. better to use sorted indices

In [3]:
function KitaevRhom_opt(J::Array{Float64,1}, N1::Int, N2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice
    Inputes: N1, N2: column and row numbers
             J: coupling constants
    Outpus: The Non-zero Hamiltonian element H[i] is in the 
            position (row[i], col[i])
            H, row, col : type: list =#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    Ham = spzeros(Float64, dim, dim)

    for tag = 0: dim-1

        for r = 0: N2-1
            for c = 0: N1-1
                ind = index(N1, N2, c, r, 0)
                
                # Sx-Sx
                next = index(N1, N2, c, r, 1)
                ntag = flip(tag, ind, next, N1, N2)
                Ham[ntag+1, tag+1] += -J[1] /4
                
                # Sy-Sy
                next = index(N1, N2, c - 1, r, 1) 
                ntag = flip(tag, ind, next, N1, N2) 
                Ham[ntag+1, tag+1] += J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                        
                #Sz-Sz
                next = index(N1, N2, c, r - 1, 1)
                Ham[tag+1, tag+1] += - J[3] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
            end
        end                       
    end
    return Ham
    end;

BenchmarkTools.Trial: 
  memory estimate:  2.03 MiB
  allocs estimate:  35
  --------------
  minimum time:     15.660 ms (0.00% GC)
  median time:      16.641 ms (0.00% GC)
  mean time:        17.001 ms (0.00% GC)
  maximum time:     20.935 ms (0.00% GC)
  --------------
  samples:          289
  evals/sample:     1

\section{Kitaev model with translation symmetry}
\subsection{Momentum basis}
add notes!

In [4]:
# Tool functions
using SparseArrays
using Arpack
using BenchmarkTools

function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    #Calculate the index of the atoms in the (r,c) unit cell
    #A-sublattice atom = 0; B-subkattice: atom = 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
    end;

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
    end;

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
    end;

function cyclebits(tag::Int, n1::Int, n2::Int, N1::Int, N2::Int)
    #=Performs a cyclic permutations of n1 steps to the right and n2 steps upward.
       Inputs: tag: tag of the reference state
               n1: cycle times of n1-direction, columns,if n1=0, no cycle in columns
               n2: cycle times of n2-direction, rows,if n2=0, no cycle in rows
               N1: number of columns
               N2: number of rows
       Outputs: ntag: the tag of the outcoming state=#
    SiteNum = N1 * N2 * 2
    ntag = 0
    for r = 0: N2-1
        for c = 0: N1-1
            for atom = 0: 1
                ind = index(N1, N2, c, r, atom)
                # notice that c,r should be positive in julia, -1%3 = -1 instead of 2
                nind = index(N1, N2, c + n1, r + n2, atom) # n1: ->right, n2 -> up
                ntag += 2^(SiteNum - nind) * bits(tag, ind, N1, N2)
            end
        end
    end
    return ntag
    end;

In [35]:
# Momentum basis generation
function MomBasis(N1::Int, N2::Int, k1::Int, k2::Int)
    #=Momentum basis list of a given k1, k2
       Inputs: N1: number of columns
               N2: number of rows    
               k1: momentum in n1-direction
               k2: momentum in n2-direction

       Outpus: mb: momentum basis
               ref: reference states
               trans: translation operation to corresponding reference state
               factor: renormalization factor 
               =#
    
    dim = 2^(N1 * N2 * 2)
    mb = [x for x = 1: dim] # momentum basis
    ref = zeros(Int, dim)
    trans = zeros(Int, dim)
    factor = -ones(dim)
    
    error = 10^(-10)
    
    k1 = -2im * pi/N1 * k1
    k2 = -2im * pi/N2 * k2
    
    for tag = 0: (dim - 1)
        count = 0 # count number of non-equicalent translation operations
        fac = 0.0 + 0.0im # accumulate renormalization factor
        if factor[tag + 1] < 0
            for n1 = 0: N1-1, n2 = 0: N2-1
                ntag = cyclebits(tag, n1, n2, N1, N2)
                ref[ntag + 1] = tag
                # note the first translation operation that obtains this state
                if factor[ntag + 1] < 0
                    trans[ntag + 1] = (N1 - n1)%N1 *10 + (N2 - n2)%N2
                end
                
                if ntag > tag
                    factor[ntag + 1] = 0
                elseif ntag == tag
                    fac += exp(k1*n1 + k2*n2)
                    count += 1
                    factor[ntag + 1] = 1
                end
            end
            if abs(fac) < error
                fac = 0
            end
            factor[tag + 1] *= abs(fac)* sqrt(N1*N2/count)
        end
    end
    
    for i = 1: dim
        mb[i] = mb[i] * sign(factor[i])
    end 
    
    factor = filter!(x->x != 0, factor)
    mb = filter!(x->x != 0, mb)
    mb = replace!(x -> x-1, mb)
    return mb, factor, trans, ref                     
    end;

In [11]:
N1 = 3; N2 = 4
k1 = 0; k2 = 0
@benchmark MomBasis(N1, N2, k1, k2)

BenchmarkTools.Trial: 
  memory estimate:  640.00 MiB
  allocs estimate:  11
  --------------
  minimum time:     8.919 s (1.54% GC)
  median time:      8.919 s (1.54% GC)
  mean time:        8.919 s (1.54% GC)
  maximum time:     8.919 s (1.54% GC)
  --------------
  samples:          1
  evals/sample:     1

\subsection{Hamiltonian matrix}
add notes

In [31]:
# Hamiltonian matrix
using SparseArrays
                           
function KitaevH(J::Array{Float64,1}, N1::Int, N2::Int, k1::Int, k2::Int, mb::Array{Int,1}
                 , factor::Array{Float64,1}, trans::Array{Int,1}, ref::Array{Int,1})
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        =#
    dim = length(mb)
    f1 = - 2 * pi / N1
    f2 = - 2 * pi / N2
    
    H_real = spzeros(Float64, dim, dim)
    H_img = spzeros(Float64, dim, dim)
    
    for tag in mb
        i = findindex(tag, mb)
        for r = 0: N2-1
            for c = 0: N1-1
                si = index(N1, N2, c, r, 0) # start index
                
                # Sz-Sz
                ni = index(N1, N2, c, r - 1, 1)
                H_real[i, i] += - J[3] * (bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)
                
                # Sx-Sx
                ni = index(N1, N2, c, r, 1) # next index
                ntag = flip(tag, si, ni, N1, N2)
                rtag = ref[ntag + 1]
                j = findindex(rtag, mb)
                if j > 0
                    l1 = div(trans[ntag+1], 10)
                    l2 = trans[ntag + 1]%10
                    
                    phase_real = cos(f1*k1*l1 + f2*k2*l2) * factor[j]/ factor[i]
                    phase_img = sin(f1*k1*l1 + f2*k2*l2) * factor[j]/ factor[i]
                    H_real[j, i] += -J[1]/4 * phase_real
                    H_img[j, i] += -J[1]/4 * phase_img
                end
             
                # Sy-Sy
                ni = index(N1, N2, c - 1, r, 1)
                ntag = flip(tag, si, ni, N1, N2)
                rtag = ref[ntag + 1]
                j = findindex(rtag, mb)
                if j > 0
                    l1 = div(trans[ntag+1], 10)
                    l2 = trans[ntag + 1]%10
                    
                    phase_real = cos(f1*k1*l1 + f2*k2*l2) * factor[j]/ factor[i]
                    phase_img = sin(f1*k1*l1 + f2*k2*l2) * factor[j]/ factor[i]
                    H_real[j, i] += J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5) * phase_real
                    H_img[j, i] += J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5) * phase_img
                end
            end
        end
    end

    H_real = (H_real + transpose(H_real))/2
    H_img = (H_img - transpose(H_img))/2
    
    H = H_real + 1im * H_img
    #H = (H + transpose(conj(H)))/2
    return H
end

KitaevH (generic function with 1 method)

In [33]:
N1 = 3; N2 = 2;
k1 = 0; k2 = 0;
J = -ones(3)
mb, factor, trans, ref = MomBasis(N1, N2, k1, k2);
@benchmark H = KitaevH(J, N1, N2, k1, k2, mb, factor, trans, ref)
#e,x = eigs(H, which = :SR)
#println(real(e/(N1*N2*2)))

BenchmarkTools.Trial: 
  memory estimate:  1.28 MiB
  allocs estimate:  82
  --------------
  minimum time:     2.005 ms (0.00% GC)
  median time:      2.319 ms (0.00% GC)
  mean time:        2.473 ms (5.65% GC)
  maximum time:     41.749 ms (94.68% GC)
  --------------
  samples:          2019
  evals/sample:     1

In [34]:
N1 = 3; N2 = 2;
k1 = 1; k2 = 1;
J = -ones(3)
mb, factor, trans, ref = MomBasis(N1, N2, k1, k2);
H = KitaevH(J, N1, N2, k1, k2, mb, factor, trans, ref)
H1 = H - conj(transpose(H))

670×670 SparseMatrixCSC{Complex{Float64},Int64} with 0 stored entries